### With predicted rental prices, we are now interested in the growth rates for each SA2

In [20]:
import pandas as pd
import glob
import os

#path = r'../data/curated/2023_2027_rental_prediction_xgboost/' # use your path
path = r'../data/curated/random_forest_pred/'
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in sorted(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

merged_df = pd.concat(li, axis=0, ignore_index=True)
predicted = merged_df.groupby(['year', 'sa2_2021'], as_index=False).agg({'predicted_price': 'mean'})
predicted

,year,sa2_2021,predicted_price
0,2023,201011001,333.279378
1,2023,201011002,313.389906
2,2023,201011005,300.917506
3,2023,201011006,335.153048
4,2023,201011007,397.992902
...,...,...,...
2490,2027,217031476,313.190592
2491,2027,217041477,296.281075
2492,2027,217041478,296.044637
2493,2027,217041479,288.989661


In [21]:
rental_2022 = pd.read_csv('../data/curated/merged_dataset/2022_merged_data.csv')
rental_2022.dropna(inplace=True)
rental_2022 = rental_2022.groupby(['year', 'sa2_2021'], as_index=False).agg({'weekly_rent': 'mean'})
rental_2022.rename({'weekly_rent': 'predicted_price'}, axis=1, inplace=True)
rental_2022
#rental_2022[rental_2022['sa2_2021']==206041127]
#set(rental_2022['sa2_2021'].unique()) - set(predicted['sa2_2021'].unique())

,year,sa2_2021,predicted_price
0,2022,201011001,437.468013
1,2022,201011002,421.270053
2,2022,201011005,402.559524
3,2022,201011006,440.000000
4,2022,201011007,530.625000
...,...,...,...
494,2022,217031476,518.750000
495,2022,217041477,414.069767
496,2022,217041478,771.301370
497,2022,217041479,437.025641


In [22]:
growth_df = pd.concat([rental_2022, predicted], axis=0).reset_index(drop=True)
#df[df['sa2_2021']==201011001]
growth_df

,year,sa2_2021,predicted_price
0,2022,201011001,437.468013
1,2022,201011002,421.270053
2,2022,201011005,402.559524
3,2022,201011006,440.000000
4,2022,201011007,530.625000
...,...,...,...
2989,2027,217031476,313.190592
2990,2027,217041477,296.281075
2991,2027,217041478,296.044637
2992,2027,217041479,288.989661


In [23]:
subset = growth_df[growth_df['sa2_2021']==201011001]
subset
growth_dict = dict()
for sa2 in growth_df['sa2_2021'].unique():
    growth_dict[sa2] = []
    subset = growth_df[growth_df['sa2_2021']==sa2]
    for i in range(len(subset['predicted_price'])-1):
        # (current year price - past year price) / past year price
        growth_rate = (subset.iloc[i+1]['predicted_price'] - subset.iloc[i]['predicted_price'])/subset.iloc[i]['predicted_price']
        growth_dict[sa2].append(growth_rate)


In [24]:
growth_dict

{201011001: [-0.2381628656722363,
  0.00971349639447407,
  0.0,
  -0.00045949712572003346,
  0.004855377971645816],
 201011002: [-0.2560831155836819, 0.0, 0.0, 0.0, 0.0],
 201011005: [-0.2524894135272724,
  1.5538385011211354e-05,
  0.006779610023374644,
  0.007140332893127173,
  0.0],
 201011006: [-0.2382885264091964,
  0.010023416901931387,
  0.004423549822374722,
  0.0020387255043491414,
  0.0005439583800686159],
 201011007: [-0.24995448388618324,
  -0.0007119324140602465,
  0.0010232144046655517,
  -0.0019151300766962578,
  -0.0005786062999468228],
 201011008: [-0.13481135521858714,
  0.01247895102893812,
  0.005418858739490341,
  0.004095230356165949,
  -0.00011199892645453461],
 201011481: [-0.19318512299887036,
  -6.641630193361968e-06,
  0.0006327387842557101,
  3.076187605485206e-05,
  -0.00040573558721830426],
 201011482: [-0.21544007597262,
  0.00845481308631406,
  0.0039804949715655695,
  0.00020210719631244228,
  -0.0005245772657921954],
 201011483: [-0.15221457641263644,


In [25]:
import numpy as np
for key in growth_dict.keys():
    growth_dict[key] = np.mean(growth_dict[key])
growth_dict

{201011001: -0.044810697686367286,
 201011002: -0.05121662311673638,
 201011005: -0.047710786445151865,
 201011006: -0.0442517751600945,
 201011007: -0.05042738765444421,
 201011008: -0.022586062804089455,
 201011481: -0.03858679991119429,
 201011482: -0.04066544759684402,
 201011483: -0.03070180922255285,
 201011484: -0.025771922776377597,
 201021009: 0.013087923464026882,
 201021010: -0.03838818666583942,
 201021011: -0.0651645301584443,
 201021012: -0.023071068304256113,
 201031013: -0.02323010001927402,
 201031014: -0.02799882966749479,
 201031015: -0.035909937580176646,
 201031016: -0.026524704586968456,
 201031017: -0.02821696369155377,
 202011018: -0.044125289340896254,
 202011019: -0.06285473575383357,
 202011020: -0.042933381858461336,
 202011021: -0.03509773882510102,
 202011022: -0.05371586713274915,
 202011023: -0.04702730572118984,
 202011024: -0.054738534044265486,
 202011025: -0.0490200114970261,
 202021026: -0.048728533927050274,
 202021027: -0.056937926870538245,
 2020